In [54]:
from pymongo import MongoClient
import requests
import zipfile
from io import BytesIO

def toInt(s):
    try: 
        return int(s)
    except ValueError:
        return s

def getDictFromText(url,id):
    r = unzip(url)
    dataArray = [line.split('\t') for line in r.splitlines()]
    colArray = dataArray.pop(0)
    result={data[id] : {col : data[idx] for idx, col in enumerate(colArray)}for data in dataArray}
    return result

def getListFromText(url):
    r = unzip(url)
    dataArray = [line.split('\t') for line in r.splitlines()]
    colArray = dataArray.pop(0)
    result=[{col : data[idx] for idx, col in enumerate(colArray)}for data in dataArray]
    return result

def unzip (url):
    request = requests.get(url, 'r')
    myzip = zipfile.ZipFile(BytesIO(request.content))
    for name in myzip.namelist():
        return myzip.read(name).decode('unicode_escape') 

client = MongoClient('ec2-52-211-76-129.eu-west-1.compute.amazonaws.com',27017)
db = client.wax_dfg
db.departement.drop()
urlRegion= 'https://www.insee.fr/fr/statistiques/fichier/2114819/reg2016-txt.zip'
dictRegion = getDictFromText(urlRegion,0)

urlDpt= 'https://www.insee.fr/fr/statistiques/fichier/2114819/depts2016-txt.zip'
listDpt = getListFromText(urlDpt)
for dpt in listDpt:
    dpt['REGION'] = dictRegion[dpt['REGION']]
db.departement.insert_many(listDpt)

